In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import seaborn as sns
import math
import gc

import sys
from pathlib import Path
d = Path().resolve().parent.parent
sys.path.insert(0, str(d))
import util.utils as utils
import util.plot_utils as plot_utils



plt.rcParams["figure.figsize"] = (12,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

from catboost import Pool, CatBoostRegressor

InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2)

ModuleNotFoundError: No module named 'catboost'

In [2]:
project_name="click-prediction"
def get_file_location(filename):
    return "../data/%s/%s" % (project_name,filename)

In [3]:
def fast_read_and_append(file_path,chunksize,fullsize):
    # in chunk reading be careful as pandas might infer a columns dtype as different for diff chunk
    # As such specifying a dtype while reading by giving params to read_csv maybe better
    # Label encoding will fail if half the rows for same column is int and rest are str
    # In case of that already happened then df_test["publisherId"] = df_test["publisherId"].apply(str)
    df = pd.DataFrame()
    iterations = 0
    total_needed_iters = math.ceil(fullsize/chunksize)
    for x in pd.read_csv(file_path, chunksize=chunksize,low_memory=False):
        print("iterations= %s out of %s" %  (iterations,total_needed_iters))
        df = pd.concat([df, x], ignore_index=True)
        iterations += 1
        gc.collect()
    return df

In [4]:
df=fast_read_and_append("../data/%s/train-step-2.csv" % project_name,1000000,63367217)


iterations= 0 out of 64
iterations= 1 out of 64
iterations= 2 out of 64
iterations= 3 out of 64
iterations= 4 out of 64
iterations= 5 out of 64
iterations= 6 out of 64
iterations= 7 out of 64
iterations= 8 out of 64
iterations= 9 out of 64
iterations= 10 out of 64
iterations= 11 out of 64
iterations= 12 out of 64
iterations= 13 out of 64
iterations= 14 out of 64
iterations= 15 out of 64
iterations= 16 out of 64
iterations= 17 out of 64
iterations= 18 out of 64
iterations= 19 out of 64
iterations= 20 out of 64
iterations= 21 out of 64
iterations= 22 out of 64
iterations= 23 out of 64
iterations= 24 out of 64
iterations= 25 out of 64
iterations= 26 out of 64
iterations= 27 out of 64
iterations= 28 out of 64
iterations= 29 out of 64
iterations= 30 out of 64
iterations= 31 out of 64
iterations= 32 out of 64
iterations= 33 out of 64
iterations= 34 out of 64
iterations= 35 out of 64
iterations= 36 out of 64
iterations= 37 out of 64
iterations= 38 out of 64
iterations= 39 out of 64
iterations

In [5]:
df_test=fast_read_and_append("../data/%s/test-step-2.csv" % project_name,1000000,24510750)

iterations= 0 out of 25
iterations= 1 out of 25
iterations= 2 out of 25
iterations= 3 out of 25
iterations= 4 out of 25
iterations= 5 out of 25
iterations= 6 out of 25
iterations= 7 out of 25
iterations= 8 out of 25
iterations= 9 out of 25
iterations= 10 out of 25
iterations= 11 out of 25
iterations= 12 out of 25
iterations= 13 out of 25
iterations= 14 out of 25
iterations= 15 out of 25
iterations= 16 out of 25
iterations= 17 out of 25
iterations= 18 out of 25
iterations= 19 out of 25
iterations= 20 out of 25
iterations= 21 out of 25
iterations= 22 out of 25
iterations= 23 out of 25
iterations= 24 out of 25
iterations= 25 out of 25


In [6]:
df_conv_status=df[df["conv_status"]==1]

In [7]:
df_fraud = df[df["Fraud"]==1]
df_conv_status.to_csv(get_file_location("train_true-step-3.csv"),index=False)
df_fraud.to_csv(get_file_location("train_fraud-step-3.csv"),index=False)
df_fraud=None
df_conv_status=None
gc.collect()

7

### Sampling - Useless

In [14]:


    
def sample_by_colname(df,colname,frac,atleast_sample_each,null_values):
    if null_values is None:
        null_values = []
    df_use = df[(~df[colname].isin(null_values)) & (df[colname].notnull())].count()
    gb = df.groupby(colname)

    
    return_df = pd.DataFrame()
#     for c in df_use[colname].unique():
#         df_temp = df_use[df_use[colname]==c]
    for x in gb.groups:
        df_temp=gb.get_group(x)
        num_elems_for_this = df_temp[colname].count()
        df_sample = pd.DataFrame()
        if num_elems_for_this < atleast_sample_each:
            df_sample=df_temp.sample(n=num_elems_for_this)
        elif num_elems_for_this*frac < atleast_sample_each:
            df_sample=df_temp.sample(n=atleast_sample_each)
        else:
            df_sample=df_temp.sample(frac=frac)
            
        return_df = pd.concat([return_df, df_sample], ignore_index=True)
    gc.collect()
    return return_df
    

    

In [15]:
df_country = sample_by_colname(df,"Country",0.001,100,None)

In [16]:
df_carrier = sample_by_colname(df,"Carrier",0.001,100,None)
gc.collect()

7

In [17]:
df_traffic = sample_by_colname(df,"TrafficType",0.001,100,None)

In [18]:
df_device = sample_by_colname(df,"Device",0.001,100,None)
# 3:50

In [19]:
df_browser = sample_by_colname(df,"Browser",0.001,100,None)

In [20]:
df_os = sample_by_colname(df,"OS",0.001,100,None)
gc.collect()

7

In [21]:
# df_refurl = sample_by_colname(df,"RefererUrl",0.00005,2,[0,1,2])

#taking refurls for not nulls
df_refurl=df[~df["RefererUrl"].isin([0,1,2])]

In [32]:
df_sub_pub = sample_by_colname(df,"subPublisherId",0.001,100,None)

In [33]:
df_advert = sample_by_colname(df,"advertiserCampaignId",0.001,100,None)

In [24]:
df_country.to_csv(get_file_location("train_country-step-3.csv"),index=False)

In [25]:
df_carrier.to_csv(get_file_location("train_carrier-step-3.csv"),index=False)

In [26]:
df_traffic.to_csv(get_file_location("train_traffic-step-3.csv"),index=False)

In [27]:
df_device.to_csv(get_file_location("train_device-step-3.csv"),index=False)

In [28]:
df_browser.to_csv(get_file_location("train_browser-step-3.csv"),index=False)

In [29]:
df_os.to_csv(get_file_location("train_os-step-3.csv"),index=False)

In [38]:
df_refurl.to_csv(get_file_location("train_refurl-step-3.csv"),index=False)

In [34]:
df_sub_pub.to_csv(get_file_location("train_sub_pub-step-3.csv"),index=False)

In [35]:
df_advert.to_csv(get_file_location("train_advert-step-3.csv"),index=False)

In [42]:
df_random = df.sample(frac=0.1)
df_random.to_csv("train_random-step-3.csv",index=False)